# The Problem with Interpolation on a Lat–Lon Grid

This notebook shows an issue that appeared during the creation of the 3D boundary conditions of the GETM setup for the 79° North Glacier fjord.
The problem is that interpolation of data between two grids does not give the expected result when the grids are in latitude and longitude.
For a physically realistic result, the data must be interpolated between two Cartesian grids.
This problem is shown here together with a possible solution.

Notebook by Markus Reinert (IOW, 2023, https://orcid.org/0000-0002-3761-8029).

In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cmocean
from scipy.interpolate import griddata
from pyproj import CRS, Transformer

## Load FESOM data

for a single point in time
and mask cells with zero salinity.

In [ ]:
fesom = xr.open_dataset("data/FESOM/salt.fesom.2010.sub.nc").isel(time=0)
fesom["salt"] = fesom.salt.where(fesom.salt > 0)
fesom

### Show data on native grid

In [ ]:
plt.title("Sea surface salinity [PSU]")
plt.scatter(fesom.lon, fesom.lat, s=1, c=np.isnan(fesom.salt.isel(nz1=0)), cmap="Greys")
plt.scatter(fesom.lon, fesom.lat, s=1, c=fesom.salt.isel(nz1=0), cmap=cmocean.cm.haline)
plt.colorbar()
plt.grid()

## Interpolate on a lat–lon grid

In [ ]:
# Choose the depth at which to interpolate
salt_layer = fesom.salt.isel(nz1=21)

# Choose the coordinates of the target grid
lon = np.arange(-17.9, -17.45, 0.025)
lat = 79.2 * np.ones_like(lon)

# Interpolate the data using lat–lon coordinates
salt = griddata((fesom.lon, fesom.lat), salt_layer, (lon, lat))

vmin = np.nanmin(salt)
vmax = np.nanmax(salt)

fig, axs = plt.subplots(3, figsize=(5, 14), dpi=200, constrained_layout=True)

ax = axs[0]
_  = ax.scatter(fesom.lon, fesom.lat, s=1, c=np.isnan(salt_layer), cmap="Greys", vmax=2)
im = ax.scatter(fesom.lon, fesom.lat, s=1, c=salt_layer, cmap=cmocean.cm.haline, vmin=vmin, vmax=vmax)
fig.colorbar(im, ax=axs, location="top", fraction=0.05, shrink=0.8, extend="both", label=f"Salinity at {salt_layer.nz1.data} m depth [PSU]")
ax.plot(lon, lat, "r-", lw=1, label="interpolation along this line")
ax.legend()

for ax in axs[1:]:
    ax.scatter(lon, lat, s=30, c=np.isnan(salt), cmap="Greys")
    ax.scatter(lon, lat, s=30, c=salt, cmap=cmocean.cm.haline, vmin=vmin, vmax=vmax)

    ax.scatter(fesom.lon, fesom.lat, s=20, c=np.isnan(salt_layer), cmap="Greys", vmax=2)
    ax.scatter(fesom.lon, fesom.lat, s=20, c=salt_layer, cmap=cmocean.cm.haline, vmin=vmin, vmax=vmax)

    ax.set_xticks(np.arange(-17.9, -17.4, 0.1))
    ax.set_xticks(np.arange(-17.9, -17.4, 0.05), minor=True)
    ax.set_yticks(np.arange(79.1, 79.3, 0.02))
    ax.set_yticks(np.arange(79.1, 79.3, 0.01), minor=True)
    ax.set_ylim(79.15, 79.25)
    ax.set_xlim(-17.9, -17.5)
    ax.grid(which="both")

axs[0].set_title("Overview")
axs[1].set_title("Aspect ratio about 1 on a Cartesian grid")
axs[2].set_title("Aspect ratio 1 on a lat–lon grid")
axs[2].set_aspect("equal")

With the aspect ratio in the middle panel, it seems wrong that some of the interpolated points are NaN (black), even though they are surrounded by valid points.
Looking at the same situation with an equal lat–lon aspect (bottom panel), it becomes clear that these points are in fact close to a masked point (grey), so they become NaN in the interpolation.

## Transform to Cartesian coordinates

### CRS of model grids

In [ ]:
crs_latlon = CRS.from_epsg(4326)
crs_latlon

### CRS for interpolation

In [ ]:
crs_cartesian = CRS.from_epsg(3413)
crs_cartesian

### CRS transformer

In [ ]:
transformer = Transformer.from_crs(crs_latlon, crs_cartesian)

### Add transformed coordinates to the dataset

In [ ]:
fesom["X"], fesom["Y"] = transformer.transform(fesom.lat, fesom.lon)
fesom.X.attrs.update({"long_name": crs_cartesian.axis_info[0].name, "units": "m", "CRS": str(crs_cartesian)})
fesom.Y.attrs.update({"long_name": crs_cartesian.axis_info[1].name, "units": "m", "CRS": str(crs_cartesian)})

### Show data on transformed grid

In [ ]:
fig, ax = plt.subplots()
_  = ax.scatter(fesom.X, fesom.Y, s=1, c=np.isnan(fesom.salt.isel(nz1=0)), cmap="Greys")
im = ax.scatter(fesom.X, fesom.Y, s=1, c=fesom.salt.isel(nz1=0), cmap=cmocean.cm.haline)
fig.colorbar(im, ax=ax)
ax.plot(*transformer.transform([79.2, 79.2, 80.3, 80.3], [-23, -15, -15, -23]), "r--", label="straight line on the map")
ax.plot(*transformer.transform(np.linspace(79.2, 79.2), np.linspace(-23, -15)), "r.", label="straight line in reality")
ax.plot(*transformer.transform(np.linspace(79.2, 80.3), np.linspace(-15, -15)), "r.")
ax.plot(*transformer.transform(np.linspace(80.3, 80.3), np.linspace(-23, -15)), "r.")
ax.legend()
ax.set_title("Sea surface salinity [PSU]")
ax.set_aspect("equal")

## Interpolate on a Cartesian grid

In [ ]:
# Choose the depth at which to interpolate
salt_layer = fesom.salt.isel(nz1=21)

# Choose the coordinates of the target grid
lon = np.arange(-20, -14.9, 0.025)
lat = 79.2 * np.ones_like(lon)

# Transform the target grid
X, Y = transformer.transform(lat, lon)

# Interpolate the data using Cartesian coordinates
salt = griddata((fesom.X, fesom.Y), salt_layer, (X, Y))

vmin = np.nanmin(salt)
vmax = np.nanmax(salt)

fig, axs = plt.subplots(ncols=2, gridspec_kw={"width_ratios": (2, 1)}, constrained_layout=True, figsize=(15, 3), dpi=300)

ax = axs[0]
ax.set_title(f"Cartesian coordinate system ({crs_cartesian})")
ax.scatter(fesom.X, fesom.Y, s=2, c=np.isnan(salt_layer), cmap="Greys", vmax=2)
ax.scatter(fesom.X, fesom.Y, s=2, c=salt_layer, cmap=cmocean.cm.haline, vmin=vmin, vmax=vmax)

_  = ax.scatter(X, Y, s=1, c=np.isnan(salt), cmap="Greys")
im = ax.scatter(X, Y, s=1, c=salt, cmap=cmocean.cm.haline, vmin=vmin, vmax=vmax)
fig.colorbar(im, ax=axs)

ax.set_xlim(510e3, 590e3)
ax.set_ylim(-1.055e6, -1.015e6)
ax.set_aspect("equal")

ax = axs[1]
ax.set_title(f"Lat–lon coordinate system ({crs_latlon})")
ax.scatter(fesom.lon, fesom.lat, s=2, c=np.isnan(salt_layer), cmap="Greys", vmax=2)
ax.scatter(fesom.lon, fesom.lat, s=2, c=salt_layer, cmap=cmocean.cm.haline, vmin=vmin, vmax=vmax)

ax.scatter(lon, lat, s=1, c=np.isnan(salt), cmap="Greys")
ax.scatter(lon, lat, s=1, c=salt, cmap=cmocean.cm.haline, vmin=vmin, vmax=vmax)

ax.set_xlim(-19, -16)
ax.set_ylim(79.0, 79.4)

With the interpolation on Cartesian coordinates, only those points are NaN (black) which are adjacent to a masked point (grey), as expected.